# ETL Steam_Games 

- En la extraccion de los archivos se pudo ver que este archivo 'Steam_games.json' no contiene datos anidados por ende la forma para extraerlos y convertilos en un DataFrame se va a realizar de la siguiente manera



##### Leer los archivos ".json" y pasarlos a un dataframe

In [1]:
# librerias
import pandas as pd
#from collections import Counter
import fun
import numpy as np
import ast
import re
from fuzzywuzzy import fuzz # libreria para caclcular similitud entre dos cadenas de texto y 'fuzz' 

s:\PROYECTO INDIVIDUAL Nº1\proyecto1\myenv\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### SE PROCEDE A TRATAR LOS DATOS

In [2]:
# abrir el csv n un DataFrame
df_games=pd.read_csv(r'./steam_games.csv ')

In [3]:
df_games.head(1)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,0.0,761140.0,Kotoshiro


- Se explicara a groso modo cada columna : 

    - **publisher&nbsp;&nbsp;&nbsp;:**&emsp;Empresa que realiza la publicacion del juego.
    - **genres&nbsp;&nbsp;&nbsp;:**&emsp;Tipo de genero del juego.
    - **app_name&nbsp;&nbsp;&nbsp;:**&emsp;Nombre del juego.
    - **title:&nbsp;&nbsp;&nbsp;:**&emsp;Nombre del juego.
    - **url:&nbsp;&nbsp;&nbsp;:**&emsp;Direccion del sitio web del juego.
    - **release_date:&nbsp;&nbsp;&nbsp;:**&emsp;Fecha de lanzamiento.
    - **tags:&nbsp;&nbsp;&nbsp;:**&emsp;Etiquetas creadas por los usuarios.
    - **reviews_url:&nbsp;&nbsp;&nbsp;:**&emsp;Direccion del sitio del review.
    - **specs:&nbsp;&nbsp;&nbsp;:**&emsp;Categoria del juego.
    - **price:&nbsp;&nbsp;&nbsp;:**&emsp;Precio en dolares.
    - **early_access:&nbsp;&nbsp;&nbsp;:**&emsp;Acceso anticipado para versiones previas al lanzamiento oficial.
    - **id:&nbsp;&nbsp;&nbsp;:**&emsp;Identificador único para el juego.
    - **developer:&nbsp;&nbsp;&nbsp;:**&emsp;Empresa que desarrollo el juego.
</br>
</br>
- Las columnas 'url', 'reviews_url' son columnas innecsarias por ende se eliminaran


In [4]:
# Verificar tipo de dato de cada columna, % de Nulos y % no Nulos y verificacion de filas completamente nulas
fun.tipo_dato(df_games)

publisher (nulos) =  8071
genres (nulos) =  3283
app_name (nulos) =  2
title (nulos) =  2050
url (nulos) =  0
release_date (nulos) =  2067
tags (nulos) =  163
reviews_url (nulos) =  2
specs (nulos) =  670
price (nulos) =  1377
early_access (nulos) =  0
id (nulos) =  2
developer (nulos) =  3299

 filas completamente nulas:  0


,Nombre_Columna,Tipo_dato,%_No_nulos,%_Nulos
0,publisher,"[<class 'str'>, <class 'float'>]",74.88,25.12
1,genres,"[<class 'str'>, <class 'float'>]",89.78,10.22
2,app_name,"[<class 'str'>, <class 'float'>]",99.99,0.01
3,title,"[<class 'str'>, <class 'float'>]",93.62,6.38
4,url,[<class 'str'>],100.00,0.00
5,release_date,"[<class 'str'>, <class 'float'>]",93.57,6.43
6,tags,"[<class 'str'>, <class 'float'>]",99.49,0.51
7,reviews_url,"[<class 'str'>, <class 'float'>]",99.99,0.01
8,specs,"[<class 'str'>, <class 'float'>]",97.92,2.08
9,price,"[<class 'str'>, <class 'float'>]",95.71,4.29


In [5]:
# Se eliminan algunas columnas innecesarias
df_games.drop(columns=['url','reviews_url'],inplace=True)

In [6]:
# Se verifican todas las filas que contengan nulos 
nulos = df_games[df_games.isnull().any(axis=1)]
nulos

,publisher,genres,app_name,title,release_date,tags,specs,price,early_access,id,developer
4,NaN,NaN,Log Challenge,NaN,NaN,"['Action', 'Indie', 'Casual', 'Sports']","['Single-player', 'Full controller support', '...",2.99,0.0,773570.0,NaN
6,NaN,"['Free to Play', 'Indie', 'Simulation', 'Sports']",SNOW - All Access Basic Pass,SNOW - All Access Basic Pass,2018-01-04,"['Free to Play', 'Indie', 'Simulation', 'Sports']","['Single-player', 'Multi-player', 'Online Mult...",9.99,0.0,774276.0,Poppermost Productions
9,RewindApp,"['Casual', 'Indie', 'Racing', 'Simulation']",Race,Race,2018-01-04,"['Indie', 'Casual', 'Simulation', 'Racing']","['Single-player', 'Multi-player', 'Partial Con...",NaN,0.0,768800.0,RewindApp
10,Qucheza,"['Action', 'Indie', 'Simulation', 'Early Access']",Uncanny Islands,Uncanny Islands,Soon..,"['Early Access', 'Action', 'Indie', 'Simulatio...",['Single-player'],NaN,1.0,768570.0,Qucheza
11,NaN,NaN,Icarus Six Sixty Six,NaN,NaN,['Casual'],"['Single-player', 'HTC Vive', 'Tracked Motion ...",Free,0.0,724910.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
32121,NaN,NaN,Kalen Chock Presents: 2017 Free Tutorial,Kalen Chock Presents: 2017 Free Tutorial,2018-01-03,"['Design & Illustration', 'Tutorial']",NaN,Free,0.0,777950.0,NaN
32123,VersoVR,"['Casual', 'Indie', 'Simulation', 'Sports', 'E...",Cricket Club,Cricket Club,January 2018,"['Early Access', 'Indie', 'Casual', 'Simulatio...",['Single-player'],NaN,1.0,772180.0,VersoVR
32125,Retro Army Limited,"['Action', 'Adventure', 'Indie']",The spy who shot me™,The spy who shot me™,2018-10-01,"['Action', 'Adventure', 'Indie']","['Single-player', 'Captions available']",NaN,0.0,771810.0,Retro Army Limited
32128,Riviysky,"['Casual', 'Indie']",BAE 2,BAE 2,2018-01-04,"['Indie', 'Casual']",['Single-player'],NaN,0.0,769330.0,Riviysky


In [7]:
# Se eliminan todas las filas con nulos 
df_games = df_games.dropna()

In [73]:
# Se revisan si existen filas duplicadas en el DataFrame 
duplicados = df_games.loc[df_games.duplicated()]
duplicados

,Id_game,App_name,Release_year,Genres,Specs,Price,Early_access,Developer


### Columna 'app_name'

In [8]:
# Verificamos duplicados en la columna 'app_name'
filas_duplicadas = fun.duplicados(df_games,'app_name')
filas_duplicadas

,publisher,genres,app_name,title,release_date,tags,specs,price,early_access,id,developer
10913,Choose Multiple LLC,"['Adventure', 'Casual', 'Indie', 'RPG', 'Simul...",Alter Ego,Alter Ego,1986-05-01,"['RPG', 'Casual', 'Indie', 'Adventure', 'Simul...","['Single-player', 'Captions available', 'Steam...",7.99,0.0,664780.0,Choose Multiple LLC
31403,Viva Media,['Adventure'],Alter Ego,Alter Ego,2010-08-03,"['Adventure', 'Point & Click', 'Mystery', 'Det...",['Single-player'],9.99,0.0,63110.0,bitComposer Games
23902,Sir Bedlam Productions,"['Adventure', 'Free to Play', 'Indie']",Black Rose,Black Rose,2016-03-12,"['Free to Play', 'Horror', 'Indie', 'Adventure...","['Single-player', 'Partial Controller Support']",Free to Play,0.0,453890.0,Sir Bedlam Productions
22898,D3 PUBLISHER,['Action'],Black Rose,Black Rose,2016-06-02,['Action'],"['Single-player', 'Downloadable Content', 'Ste...",0.99,0.0,464510.0,TAMSOFT
4536,Badland Games,"['Action', 'Casual', 'Indie']",Castles,Castles,2015-10-09,"['Casual', 'Indie', 'Action', 'Puzzle', 'Local...","['Single-player', 'Shared/Split Screen', 'Stea...",2.99,0.0,384010.0,Whootgames
16798,interplay,"['Adventure', 'Strategy']",Castles,Castles,1991-01-02,"['Adventure', 'Strategy']",['Single-player'],9.99,0.0,666660.0,Quicksilver Software
12788,eugeniy_kh,"['Indie', 'Simulation', 'Strategy']",Colony,Colony,2017-12-15,"['Indie', 'Simulation', 'Strategy']",['Single-player'],1.99,0.0,720060.0,eugeniy_kh
20305,Varitech,"['Casual', 'Simulation']",Colony,Colony,2016-11-29,"['Casual', 'Simulation']","['Single-player', 'Partial Controller Support']",0.99,0.0,509450.0,Jamie Cunningham
1665,Iceberg Interactive,"['Action', 'Indie']",Dark Matter,Dark Matter,2013-10-17,"['Action', 'Indie', 'Side Scroller', 'Sci-fi',...","['Single-player', 'Steam Achievements', 'Full ...",9.99,0.0,251410.0,InterWave Studios
27260,Meridian4,"['Action', 'Casual', 'Indie']",Dark Matter,Dark Matter,2015-02-27,"['Casual', 'Action', 'Indie', ""Shoot 'Em Up"", ...","['Single-player', 'Partial Controller Support']",4.99,0.0,345130.0,Meridian4


Se observan varios duplicados por nombre pero al verificar mas a fondo se observa qu eson de diferentes desarrolladores excepto por el ultimo juego llamado 'Wolfenstein II: The New Colossus' ya que dicho juego si se ve duplicado.

In [9]:
#se elimina uno de los dos duplicados teniendo como base la id 
df_games=df_games.drop_duplicates(subset='id', keep='first')

In [10]:
df_games.head(3)

,publisher,genres,app_name,title,release_date,tags,specs,price,early_access,id,developer
0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",['Single-player'],4.99,0.0,761140.0,Kotoshiro
1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...","['Single-player', 'Multi-player', 'Online Mult...",Free To Play,0.0,643980.0,Secret Level SRL
2,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...","['Single-player', 'Multi-player', 'Online Mult...",Free to Play,0.0,670290.0,Poolians.com


- Se eliminara la columna de 'publisher' ya que no es necesaria es mas importante 'developer' puesto que es quien desarrollo el video juego mientras que 'publisher' es la empresa que publica.
- En app_name y title parece tener coincidencias.


### Columnas 'app_name - tittle'

In [11]:
# Función para calcular el porcentaje de coincidencia entre dos cadenas
def calcular_similitud(cadena1, cadena2):
    return fuzz.ratio(cadena1, cadena2)


In [12]:
# Aplicar la función y crear una nueva columna con los porcentajes de coincidencia
df_games['Porcentaje_Coincidencia'] = df_games.apply(lambda row: calcular_similitud(row['app_name'], row['title']), axis=1)

In [13]:
# Dimension del DF
df_games.shape

(22510, 12)

In [14]:
# se realiza una busqueda de cada valor unico en la nueva columna para ver cuantos % de coincidensia salen
x = df_games['Porcentaje_Coincidencia'].value_counts()
x

Porcentaje_Coincidencia
100    22159
96        87
95        76
94        51
97        33
93        27
92        11
88        11
89        11
91        10
90         9
87         6
98         4
86         4
85         4
82         3
83         2
43         1
78         1
Name: count, dtype: int64

- Existe un alto % de coincidencias entre 'app_name' y 'title' por ende se eliminara 'title'
- La columna 'tags' hace referencia a etiquetas creadas por los jugadores, no se evaluara ya que un tag creado por alguien externo podria generar confusion, es decir tengo un juego de categoria shooter y varios usuarios pusieron tags de "Racing, space, romance, etc" son etiquetas que no corresponden a lo que si el juego es por tal motivo no se tomara en cuenta y se eliminara, tambien se eliminara la columna creada anteriormente 'porcentaje....'.

In [15]:
# Se procede a eliminar la columna 'tittle' y dejar 'appname', se eliminan demas columnas innecesarias
df_games.drop(columns=['title','publisher','tags','Porcentaje_Coincidencia'],inplace=True)

In [16]:
df_games.columns

Index(['genres', 'app_name', 'release_date', 'specs', 'price', 'early_access',
       'id', 'developer'],
      dtype='object')

### Columna 'id'

In [17]:
# Cambiar nombre a la columna 'id' y volverla a tipo 'int' 
df_games.rename(columns={'id' : 'id_game'}, inplace=True)
df_games['id_game'] = df_games['id_game'].astype(int)

### Columna 'price'

In [18]:
df_games['price'].unique()

array(['4.99', 'Free To Play', 'Free to Play', '0.99', '3.99', '18.99',
       '29.99', '10.99', '2.99', '1.5899999999999999', '14.99', '1.99',
       '59.99', '9.99', '6.99', '7.99', '39.99', 'Free', '19.99', '7.49',
       '8.99', '12.99', '5.99', '2.49', '15.99', '1.25', '24.99', '17.99',
       '61.99', '3.49', '11.99', '13.99', 'Free Demo', 'Play for Free!',
       '34.99', '1.49', '32.99', '99.99', '14.95', '69.99', '16.99',
       '79.99', '49.99', '13.98', '149.99', '771.71', 'Install Now',
       '21.99', '89.99', 'Play WARMACHINE: Tactics Demo', '0.98',
       '139.92', '4.29', 'Free Mod', '54.99', '64.99', '74.99', '0.89',
       '0.5', 'Play Now', '299.99', '1.29', '119.99', '44.99', '3.0',
       '15.0', '1.3900000000000001', 'Free HITMAN™ Holiday Pack', '2.0',
       '4.0', '1.9500000000000002', '1.5', '6.66', '26.99', '399.99',
       '31.99', '20.0', '40.0', '5.0', '3.33', '38.85', '71.7', '995.0',
       '5.49', '27.49', '3.39', '6.0', '19.95', '20.99', '499.99',
     

- Se observan muchos valores numericos distintos, y tambien precios algo elevados como por ejemplo = '771.71' o '995.0', se verificaran algunos casos particulares.

In [19]:
## son juegos gratuitos
df_games[(df_games['price']=='Free To Play') | (df_games['price'] == 'Free to Play!')| (df_games['price'] == 'Play for Free!')| (df_games['price'] == 'Free')]

,genres,app_name,release_date,specs,price,early_access,id_game,developer
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-01-04,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,0.0,643980,Secret Level SRL
60,"['Casual', 'Indie']",Zen of Sudoku,2006-12-14,['Single-player'],Free,0.0,4900,Unknown Worlds Entertainment
191,['Action'],Mare Nostrum,2008-10-17,"['Multi-player', 'Mods', 'Steam Achievements']",Free,0.0,1230,Sandstorm Productions
248,"['Action', 'Simulation']",Darkest Hour: Europe '44-'45,2009-06-09,"['Multi-player', 'Mods', 'Partial Controller S...",Free,0.0,1280,Darklight Games
356,['Action'],Defence Alliance 2,2009-10-22,"['Single-player', 'Multi-player', 'Co-op', 'Mo...",Free,0.0,35420,Defence Alliance Team
...,...,...,...,...,...,...,...,...
31969,"['Action', 'Free to Play']",SCP: Secret Laboratory,2017-12-29,['Online Multi-Player'],Free,0.0,700330,Hubert Moszka
31993,"['Free to Play', 'Simulation']",Six Days of Snow,2017-12-29,"['Single-player', 'Steam Achievements']",Free,0.0,754810,ebi-hime
32007,"['Action', 'Casual', 'Free to Play', 'Indie']",Bloodworks,2018-01-01,"['Single-player', 'Steam Achievements', 'Steam...",Free,0.0,717990,Engin Mercan
32100,"['Action', 'Free to Play', 'Massively Multipla...",忍者村大战2,2018-01-03,"['Online Multi-Player', 'In-App Purchases']",Free To Play,0.0,754350,杭州分浪网络科技有限公司


In [20]:
# son juegos
df_games[(df_games['price']=='Free HITMAN™ Holiday Pack')| (df_games['price'] == 'Play WARMACHINE: Tactics Demo')| (df_games['price'] == 'Third-party')]

,genres,app_name,release_date,specs,price,early_access,id_game,developer
2871,"['Indie', 'Strategy']",WARMACHINE: Tactics,2014-11-20,"['Single-player', 'Multi-player', 'Cross-Platf...",Play WARMACHINE: Tactics Demo,0.0,253510,WhiteMoon Dreams
5589,['Action'],HITMAN™,2016-03-11,"['Single-player', 'Steam Achievements', 'Full ...",Free HITMAN™ Holiday Pack,0.0,236870,"IO Interactive A/S,Feral Interactive (Linux),F..."
31838,['Casual'],Peggle Extreme,2007-09-11,['Single-player'],Third-party,0.0,3483,"PopCap Games, Inc."


In [21]:
# son juegos
df_games[(df_games['price']=='Free Demo')| (df_games['price'] == 'Free Mod')]

,genres,app_name,release_date,specs,price,early_access,id_game,developer
920,['Indie'],The White Laboratory Demo,2012-04-06,['Single-player'],Free Demo,0.0,213530,TAO
921,"['Indie', 'Casual']",Atooms to Moolecules Demo,2012-04-06,['Single-player'],Free Demo,0.0,213390,BitSits Games
3314,"['Action', 'Free to Play', 'RPG']",Modular Combat,2008-01-18,"['Multi-player', 'Co-op', 'Captions available'...",Free Mod,0.0,349480,Ascension Entertainment LLC
13947,"['Action', 'Free to Play', 'RPG']",Base Defense,2012-10-31,"['Single-player', 'Multi-player', 'Co-op']",Free Mod,0.0,632730,PLut
28314,['Action'],Red Orchestra 2/Rising Storm Beta Community Maps,2014-09-23,"['Single-player', 'Multi-player', 'Steam Achie...",Free Mod,0.0,324850,"RO2/RS Community,Tripwire Interactive"
28315,['Action'],Red Orchestra 2/Rising Storm Alpha Community Maps,2014-09-23,"['Single-player', 'Multi-player', 'Steam Achie...",Free Mod,0.0,324830,"RO2/RS Community,Tripwire Interactive"
30910,['Indie'],Plutonic Repulse Demo,2012-06-12,"['Single-player', 'Game demo']",Free Demo,0.0,213490,Blackcow Studios


In [22]:
# son juegos
df_games[(df_games['price']=='Play the Demo')| (df_games['price'] == 'Play Now')]

,genres,app_name,release_date,specs,price,early_access,id_game,developer
4026,"['Casual', 'Indie']",Oblivious Garden ~White Day,2015-07-20,"['Single-player', 'Downloadable Content', 'Ste...",Play Now,0.0,345040,"CorypheeSoft,DigitalEZ"
22734,['Strategy'],Legends of Callasia,2016-06-10,"['Single-player', 'Multi-player', 'Online Mult...",Play the Demo,0.0,438920,Boomzap Entertainment


In [23]:
# Es una aplicacion y no un juego 
df_games[(df_games['price']=='Install Now')]

,genres,app_name,release_date,specs,price,early_access,id_game,developer
2405,['Utilities'],EVGA Precision XOC,2014-09-19,"['Single-player', 'Steam Achievements']",Install Now,0.0,268850,EVGA


In [25]:
# Se eliminan el que es una aplicacion
df_games.drop([2405], inplace=True)

- Se Procede a reemplazar los tipo str por 0 es decri los que tienen palabras en vez de numeros, esto se hace dado que se necesita trabajar a futuro con esta columna 

In [28]:
# se cambian los datos no numericos a valores 0.00 y que los valores flotantes queden redondeados y con 2 decimales
reemplazar = ['Free To Play', 'Free to Play', 'Free','Free Demo', 'Play for Free!','Play WARMACHINE: Tactics Demo','Free Mod', 'Play Now',
              'Free HITMAN™ Holiday Pack','Play the Demo','Third-party']

valor = 0.00
for valores in reemplazar:
    df_games['price'] = df_games['price'].replace(valores, valor)

In [30]:
# verificar los dos datos faltantes en 'price'
df_games[df_games[['price']].isna().any(axis=1)]

,genres,app_name,release_date,specs,price,early_access,id_game,developer


In [32]:
# Convertir la columna 'price' a tipo de dato float
df_games['price'] = df_games['price'].astype(float)

In [33]:
df_games['price'].unique()

array([4.9900e+00, 0.0000e+00, 9.9000e-01, 3.9900e+00, 1.8990e+01,
       2.9990e+01, 1.0990e+01, 2.9900e+00, 1.5900e+00, 1.4990e+01,
       1.9900e+00, 5.9990e+01, 9.9900e+00, 6.9900e+00, 7.9900e+00,
       3.9990e+01, 1.9990e+01, 7.4900e+00, 8.9900e+00, 1.2990e+01,
       5.9900e+00, 2.4900e+00, 1.5990e+01, 1.2500e+00, 2.4990e+01,
       1.7990e+01, 6.1990e+01, 3.4900e+00, 1.1990e+01, 1.3990e+01,
       3.4990e+01, 1.4900e+00, 3.2990e+01, 9.9990e+01, 1.4950e+01,
       6.9990e+01, 1.6990e+01, 7.9990e+01, 4.9990e+01, 1.3980e+01,
       1.4999e+02, 7.7171e+02, 2.1990e+01, 8.9990e+01, 9.8000e-01,
       1.3992e+02, 4.2900e+00, 5.4990e+01, 6.4990e+01, 7.4990e+01,
       8.9000e-01, 5.0000e-01, 2.9999e+02, 1.2900e+00, 1.1999e+02,
       4.4990e+01, 3.0000e+00, 1.5000e+01, 1.3900e+00, 2.0000e+00,
       4.0000e+00, 1.9500e+00, 1.5000e+00, 6.6600e+00, 2.6990e+01,
       3.9999e+02, 3.1990e+01, 2.0000e+01, 4.0000e+01, 5.0000e+00,
       3.3300e+00, 3.8850e+01, 7.1700e+01, 9.9500e+02, 5.4900e

- Los valores se ven en forma de notacion cientifica por ende se procede a cambiar la configuracion usando la libreria de numpy

In [34]:
np.set_printoptions(suppress=False) # Configurar NumPy para suprimir la notación científica
np.set_printoptions(formatter={'float': '{:.2f}'.format}) # Configurar NumPy para mostrar los valores con dos decimales

In [35]:
df_games['price'].unique()

array([4.99, 0.00, 0.99, 3.99, 18.99, 29.99, 10.99, 2.99, 1.59, 14.99,
       1.99, 59.99, 9.99, 6.99, 7.99, 39.99, 19.99, 7.49, 8.99, 12.99,
       5.99, 2.49, 15.99, 1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99,
       34.99, 1.49, 32.99, 99.99, 14.95, 69.99, 16.99, 79.99, 49.99,
       13.98, 149.99, 771.71, 21.99, 89.99, 0.98, 139.92, 4.29, 54.99,
       64.99, 74.99, 0.89, 0.50, 299.99, 1.29, 119.99, 44.99, 3.00, 15.00,
       1.39, 2.00, 4.00, 1.95, 1.50, 6.66, 26.99, 399.99, 31.99, 20.00,
       40.00, 5.00, 3.33, 38.85, 71.70, 995.00, 5.49, 27.49, 3.39, 6.00,
       19.95, 20.99, 499.99, 27.99, 199.99, 4.68, 131.40, 44.98, 202.76,
       2.30, 0.95, 36.99, 172.24, 249.99, 2.97, 10.96, 2.66, 6.48, 10.00,
       1.00, 11.15, 49.00, 199.00, 99.00, 87.94, 0.49, 9.98, 9.95, 12.89,
       6.49, 1.87, 41.99, 4.49, 289.99, 23.96, 5.65, 12.00, 129.99, 23.99,
       13.37, 189.96, 59.95, 160.91])

In [40]:
# Se verifican los precios altos
precios_altos = df_games[df_games['price']>100]
precios_altos = precios_altos.sort_values(by='price', ascending=False)
precios_altos

,genres,app_name,release_date,specs,price,early_access,id_game,developer
12088,"['Animation &amp; Modeling', 'Design &amp; Ill...",Strata Spaces VR – Professional Edition Upgrade,2017-10-26,"['Downloadable Content', 'Steam Achievements',...",995.00,0.0,719000,"Strata Mixed Reality, Inc."
2392,['Simulation'],X-Plane 10 Global - 64 Bit,2014-07-14,"['Single-player', 'Multi-player', 'Cross-Platf...",771.71,0.0,292180,Laminar Research
17794,"['Free to Play', 'Massively Multiplayer', 'RPG']",EVE Online: 15400 PLEX,2017-05-09,"['Online Multi-Player', 'MMO', 'Downloadable C...",499.99,0.0,629796,CCP
14304,"['Animation &amp; Modeling', 'Design &amp; Ill...",CUR3D Maker Edition - Permission for commercia...,2017-11-06,['Downloadable Content'],499.99,0.0,691560,RUHRSOURCE GmbH
9074,"['Animation &amp; Modeling', 'Design &amp; Ill...",GameMaker Studio 2 Mobile,2017-03-08,['In-App Purchases'],399.99,0.0,585620,YoYo Games Ltd.
9075,"['Animation &amp; Modeling', 'Design &amp; Ill...",GameMaker Studio 2 UWP,2017-03-08,['In-App Purchases'],399.99,0.0,585610,YoYo Games Ltd.
4445,"['Animation &amp; Modeling', 'Design &amp; Ill...",MODO indie 10,2015-09-24,['Steam Cloud'],299.99,0.0,401090,The Foundry
29566,"['Animation &amp; Modeling', 'Education', 'Uti...",Clickteam Fusion 2.5 Developer Upgrade,2014-01-27,"['Single-player', 'Multi-player', 'MMO', 'Co-o...",299.99,0.0,267810,Clickteam
26852,"['Animation &amp; Modeling', 'Design &amp; Ill...",CopperCube - Professional Edition,2015-04-24,"['Single-player', 'Downloadable Content', 'Sta...",289.99,0.0,366190,Ambiera e.U.
17793,"['Free to Play', 'Massively Multiplayer', 'RPG']",EVE Online: 7430 PLEX,2017-05-09,"['Online Multi-Player', 'MMO', 'Downloadable C...",249.99,0.0,629795,CCP


Se realiza trazabilidad de los top 5 items mas caros y se evidensio lo siguiente: 
Los precios son reales exceptuando por 'X-Plane 10 Global - 64 Bit' ya que el precio no concuerda con el historico de precio que se encuentra en steamdb, algunas busquedas se dejaran acontinuacion: https://steamdb.info/app/719000/   -   https://steamdb.info/app/629796/  -   https://steamdb.info/app/691560/   -  https://steamdb.info/app/585620/

El item que no concuerda su precio : https://steamdb.info/app/292180/

In [ ]:
# Se eliminan 'X-Plane 10 Global - 64 Bit'
df_games.drop([2392], inplace=True)

### Columna 'early_access'

In [46]:
# Se transforma los valores booleanos en binario
df_games['early_access'] = df_games['early_access'].astype('Int8')

### Columna 'release_date'

In [48]:
# se crea un ciclo for para poder ver todos los datos de la columna 'release_date' y abrirlo en un editor de texto 
fechas = df_games['release_date'].unique()
for fecha in fechas:
    print(fecha)

2018-01-04
2017-07-24
2017-12-07
2018-01-03
2017-12-22
2017-12-23
1997-06-30
1998-11-08
2016-11-25
2018-01-01
2006-07-06
2006-07-11
2017-12-29
2017-12-30
2005-08-09
2006-09-29
2006-11-20
2006-11-29
2006-11-24
2006-12-14
2006-12-19
2003-08-23
2006-12-21
2006-04-17
2006-08-01
2005-07-12
2007-06-26
2006-07-24
2002-11-12
2000-11-17
2003-10-23
2006-10-16
1998-10-31
1995-04-01
2007-06-05
2006-05-23
2006-10-17
1996-06-17
2006-12-20
1995-04-30
1995-06-01
1994-05-05
1994-08-03
2001-11-20
1997-02-28
1993-10-10
2003-09-09
2007-08-03
2007-05-31
2007-03-20
2006-01-01
2007-08-21
2006-10-09
2004-09-22
2006-06-26
2007-12-14
2004-04-20
2003-07-01
2003-10-14
2006-04-07
2001-07-25
2008-10-28
2007-10-10
2007-10-23
2007-08-01
2007-09-01
2007-12-03
2008-02-08
2007-05-08
2006-05-02
2007-10-16
2006-05-30
2006-10-26
2005-03-14
2003-02-03
1998-04-30
2008-03-21
2003-02-18
2004-03-23
2000-10-25
2006-12-12
2005-03-15
2008-05-28
2008-06-04
2008-06-10
2005-12-08
2007-11-13
2007-03-27
2006-12-15
2008-03-24
2005-09-15

In [50]:
# se verifica este dato ya que tiene una fecha extraña 
df_games[(df_games['release_date']=='SOON™')]


,genres,app_name,release_date,specs,price,early_access,id_game,developer
32008,"['Action', 'Indie']",DUSK,SOON™,"['Single-player', 'Online Multi-Player', 'Stea...",20.0,0,519860,David Szymanski


In [52]:
df_games.shape

(22508, 8)

In [51]:
# se verifica el tipo de dato en 'release_date'
print(df_games['release_date'][0])
print(type(df_games['release_date'][0]))

2018-01-04
<class 'str'>


In [53]:
# Se filtra y muestra los valores únicos que contienen letras
df_games['release_date'].loc[df_games['release_date'].apply(lambda x: isinstance(x, str) and any(c.isalpha() for c in x))].unique()
# primero se verifica si cada elemento es de tipo 'str'n y luego con 'c.isalpha() for c in x' Comprueba si en x hay al menos un carácter que sea una letra (alfabético). 

array(['Jun 2009', 'Oct 2010', 'Feb 2011', 'Sep 2014', 'Apr 2016',
       'Jul 2017', 'SOON', 'Apr 2017', 'Jan 2017', 'Nov 2016', 'Oct 2016',
       'Jul 2016', 'Jun 2016', 'Aug 2015', 'Jun 2015', 'May 2015',
       'Jan 2015', 'Nov 2014', 'Aug 2014', 'Jul 2014', 'May 2014',
       'Feb 2013', 'Dec 2012', 'Jul 2010', 'Mar 2010', 'Jan 2010',
       'Oct 2009', 'Sep 2009', 'SOON™'], dtype=object)

- Como es necesario tener solo el año de lanzamiento se procedera a solo tomar los datos que tengan el año

In [54]:
# Se filtra y extrae solo los años 
df_games['release_year'] = df_games['release_date'].apply(lambda x: re.findall(r'\d{4}', str(x))[0] if isinstance(x, str) and re.match(r'\d{4}', str(x)) else None)

In [56]:
# Se elimina la columna 'release_date'
df_games.drop(columns='release_date', inplace=True)

In [57]:
# Se buscan los valores vacios en 'release_year'
df_games[df_games[['release_year']].isna().any(axis=1)]

,genres,app_name,specs,price,early_access,id_game,developer,release_year
250,"['Adventure', 'Indie']",Blueberry Garden,['Single-player'],4.99,0,29160,Erik Svedäng,None
506,"['RPG', 'Strategy']",Age of Wonders II: The Wizard's Throne,"['Single-player', 'Multi-player']",9.99,0,61510,Triumph Studios,None
509,"['RPG', 'Strategy']",Age of Wonders Shadow Magic,"['Single-player', 'Multi-player']",9.99,0,61520,Triumph Studios,None
510,"['RPG', 'Strategy']",Age of Wonders,"['Single-player', 'Multi-player']",5.99,0,61500,Triumph Studios,None
599,['Racing'],Test Drive Unlimited 2,"['Single-player', 'Multi-player', 'Co-op', 'St...",19.99,0,9930,Eden Studios,None
2607,"['Action', 'Indie', 'Racing', 'Simulation', 'S...",Altitude0: Lower & Faster,"['Single-player', 'Multi-player', 'Online Mult...",18.99,1,308080,Gugila,None
6004,"['Action', 'Adventure', 'Casual', 'Indie', 'St...",Magma Chamber,"['Single-player', 'Steam Achievements', 'Steam...",4.99,1,463240,IndieKeep Games,None
10563,"['Indie', 'Early Access']",Juke,"['Single-player', 'Full controller support']",4.99,1,665780,Jello Godfri,None
13034,"['Adventure', 'Casual', 'Indie']",Franky the Bumwalker,['Single-player'],0.99,0,761310,Onlyjoy`s production,None
18220,"['Action', 'Casual', 'Indie', 'Sports', 'Early...",Tuebor: I Will Defend,"['Multi-player', 'Online Multi-Player', 'Co-op...",24.99,1,521330,"Strength in Numbers Studios, Inc",None


In [58]:
# Reemplazar valores nulos por 'No_Data'
df_games['release_year'] = df_games['release_year'].fillna("No_Data")

In [59]:
df_games['release_year'].unique()

array(['2018', '2017', '1997', '1998', '2016', '2006', '2005', '2003',
       '2007', '2002', '2000', '1995', '1996', '1994', '2001', '1993',
       '2004', '2008', '2009', 'No_Data', '1999', '1992', '1989', '2010',
       '2011', '2013', '2012', '2014', '1983', '1984', '2015', '1990',
       '1988', '1991', '1987', '1986', '2021', '2019', '1985'],
      dtype=object)

In [60]:
df_games.head(1)

,genres,app_name,specs,price,early_access,id_game,developer,release_year
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,['Single-player'],4.99,0,761140,Kotoshiro,2018


### Columna 'Specs'


In [62]:
# Cilco for para poder ver todos los valores de la columna en un texto
especif = df_games['specs'].unique()
for i in especif:
    print(i)

['Single-player']
['Single-player', 'Multi-player', 'Online Multi-Player', 'Cross-Platform Multiplayer', 'Steam Achievements', 'Steam Trading Cards', 'In-App Purchases']
['Single-player', 'Multi-player', 'Online Multi-Player', 'In-App Purchases', 'Stats']
['Single-player', 'Steam Achievements']
['Single-player', 'Multi-player', 'Online Multi-Player', 'Cross-Platform Multiplayer', 'Downloadable Content', 'Steam Achievements', 'Full controller support', 'Steam Trading Cards', 'In-App Purchases', 'Steam Cloud', 'Steam Leaderboards']
['Single-player', 'Steam Achievements', 'Steam Cloud']
['Single-player', 'Downloadable Content', 'Steam Achievements', 'Steam Cloud']
['Single-player', 'Downloadable Content', 'Steam Achievements', 'Partial Controller Support', 'Steam Leaderboards']
['Single-player', 'Cross-Platform Multiplayer', 'Downloadable Content', 'Partial Controller Support', 'Steam Leaderboards']
['Single-player', 'Local Co-op', 'Shared/Split Screen', 'Partial Controller Support']
['Si

Esta columna muestra como el tipo de categoria del juego es decir si es de un solo jugador o multijugador o coperativo , pvp, etc podria servir para el modelo de ML

In [66]:
# # Capitalizar la primera letra de cada palabra en el nombre de las columnas
def capitalizar(texto):
    return texto.capitalize()
df_games.columns = df_games.columns.map(capitalizar)

In [67]:
df_games.columns

Index(['Genres', 'App_name', 'Specs', 'Price', 'Early_access', 'Id_game',
       'Developer', 'Release_year'],
      dtype='object')

In [68]:
# se ordenan las columnas a una manera mas legible
orden=['Id_game','App_name','Release_year','Genres','Specs','Price','Early_access','Developer']
df_games = df_games[orden]

In [69]:
# Ordenamos los id's en forma ascendente y hacemos un reset al index 
df_games=df_games.sort_values(by='Id_game', ascending=True)
df_games.reset_index(drop=True, inplace=True)

In [70]:
df_games.head(2)

,Id_game,App_name,Release_year,Genres,Specs,Price,Early_access,Developer
0,10,Counter-Strike,2000,['Action'],"['Multi-player', 'Valve Anti-Cheat enabled']",9.99,0,Valve
1,20,Team Fortress Classic,1999,['Action'],"['Multi-player', 'Valve Anti-Cheat enabled']",4.99,0,Valve


In [71]:
# Exportamos a csv
df_games.to_csv('games.csv',index=False)